In [22]:
import os
from pathlib import Path
import zipfile
import urllib
import opendatasets as od
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression

In [26]:
path = 'data/credit-data.csv'
isExisting = os.path.exists(path)

if not isExisting:
    od.download(
        "https://www.kaggle.com/datasets/kelvinkelue/credit-card-fraud-prediction/download?datasetVersionNumber=1")
    Path("data/").mkdir(parents=True, exist_ok=True)
    !mv "credit-card-fraud-prediction/fraud test.csv" "data/credit-data.csv"
    !rm "credit-card-fraud-prediction"

In [28]:
file= 'data/credit-data.csv'
fraud_data = pd.read_csv(file)

In [29]:

#Usuwamy kolumne z liczba porzadkowa
fraud_data.drop('Unnamed: 0', axis=1, inplace=True)

#Tu printujemy analizę danych, przyda się przy opracowywaniu modelu

#Zwracamy typy danych dla każdej kolumny, przydatne przy przygotowywaniu danych
print("----------------------Typy danych")
print(fraud_data.dtypes) 

#Zwraca statystyki dla kolumny numerycznych jak średnia mediana itp, to może się przydać do oczyszczania danych numerycznych
print("----------------------Statystyki dla kolumn numerycznych")
print(fraud_data.describe()) 

#Tu przekształcamy kolumny z typów ogólnych na takie, które mogą być użyte przy uczeniu

#konwertuję datę najpierw na datetime, a potem datetime przekształcam na typ numeryczny
columns_trans_date = pd.to_numeric(pd.to_datetime(fraud_data['trans_date_trans_time']))

## TODO: Trzeba zrobić tak dla reszty kolumn, które na ten moment są dropnięte, część pewnie trzeba podzielić na kategoryczne, część, zgrupować, a części może faktycznie się pozbyć 


#Usuwamy niepotrzebne już kolmuny (lub wywołujące teraz błędy)
fraud_data.drop('trans_date_trans_time', axis=1, inplace=True)
fraud_data.drop('merchant', axis=1, inplace=True)
fraud_data.drop('category', axis=1, inplace=True)
fraud_data.drop('first', axis=1, inplace=True)
fraud_data.drop('last', axis=1, inplace=True)
fraud_data.drop('gender', axis=1, inplace=True)
fraud_data.drop('street', axis=1, inplace=True)
fraud_data.drop('city', axis=1, inplace=True)
fraud_data.drop('state', axis=1, inplace=True)
fraud_data.drop('job', axis=1, inplace=True)
fraud_data.drop('dob', axis=1, inplace=True)
fraud_data.drop('trans_num', axis=1, inplace=True)

#dodajemy przetworzone kolumny
fraud_data = pd.concat([fraud_data,columns_trans_date], axis=1)

toPredict = fraud_data['is_fraud']
fraud_data.drop('is_fraud', axis=1, inplace=True)

#Definiujemy użyty model, w tym wypadku regresja logistyczna
model = LogisticRegression(max_iter=1000)

#To teoretycznie oblicza wyniki uczenia, jako estymator daję model, dane i zmienną do przewidzenia, cv to definicja że ma być użyte 10-fold, oraz wynik na jakim się skupiamy to dokładność
## TODO chociaż coś tu jest ewidentnie nie tak, wywaliłem prawie wszystko i mam 99% dokładności xD Nie wiem o co tu chodzi ale trzeba sprawdzić
scores = cross_val_score(model, fraud_data, toPredict, cv=10, scoring='accuracy')
print(scores)

----------------------Typy danych
trans_date_trans_time     object
cc_num                   float64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
job                       object
dob                       object
trans_num                 object
unix_time                  int64
merch_lat                float64
merch_long               float64
is_fraud                   int64
dtype: object
----------------------Statystyki dla kolumn numerycznych
             cc_num            amt            zip            lat  \
count  5.557190e+05  555719.000000  555719.000000  555719.000000   
mean   4.178387e+17      69.39281

/tmp/ipykernel_12459/2226741863.py:17: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  columns_trans_date = pd.to_numeric(pd.to_datetime(fraud_data['trans_date_trans_time']))


[0.99614914 0.99614914 0.99614914 0.99614914 0.99613115 0.99613115
 0.99613115 0.99613115 0.99613115 0.99614907]
